In [ ]:
%pip install tensorflow
%pip install pydot

In [ ]:
# Mount the current directory
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Split the data into training and testing sets
# 70% for training and 15% for testing and 15% for validation
import os, shutil, pathlib
from pathlib import Path

original_dir: Path = pathlib.Path.cwd() / 'Classifier'
new_base_dir: Path = pathlib.Path.cwd() / 'Classifier/trash_classifier_pre_processed'

CATEGORIES: list[str] = ['Vegetation', 'Textile Trash', 'Plastic',\
                        'Paper', 'Miscellaneous Trash', 'Metal', 'Glass',\
                        'Food Organics', 'Cardboard'
                        ]

def get_length_of_category(path: Path) -> int:
    return len([f for f in path.iterdir() if f.is_file() and f.suffix == '.jpg'])

def make_subset(subset_name: str, start_ratio: float, end_ratio: float, cat_len: dict[str,int]) -> None:
    for category in CATEGORIES:
        dir: Path = new_base_dir / subset_name / category
        try:
          os.makedirs(dir)
        except OSError:
          raise OSError(f"Directory {dir} already exists.")
        total_files: int = cat_len[category]
        start_idx: int = int(start_ratio * total_files)+1
        end_idx: int = int(end_ratio * total_files)+1
        end_idx: int = min(end_idx, total_files)
        fnames: list[str] = [f"{category}_{i}.jpg" for i in range(start_idx, end_idx)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / category / fname, dst=dir / fname)

In [ ]:
print(original_dir)

/content/Classifier


In [ ]:
import tensorflow as tf
import keras
from pathlib import Path

curr_dir: Path = Path('/content/drive/MyDrive/Colab Notebooks/Classifier/')

def get_size_directory(path: Path) -> int:
    return len([f for f in path.iterdir() if f.is_file() and f.suffix == '.jpg'])

def dataset_categories(base_dir: str) -> tf.data.Dataset:
    dataset: tf.data.Dataset = keras.utils.image_dataset_from_directory(
        str(curr_dir / 'trash_classifier_pre_processed' / base_dir),
        labels='inferred',
        image_size=(224, 224),
        batch_size=32,
    )
    return dataset

train_dataset: tf.data.Dataset = dataset_categories('train')

test_dataset: tf.data.Dataset = dataset_categories('test')

validation_dataset: tf.data.Dataset = dataset_categories('val')

print(train_dataset)

Found 9018 files belonging to 9 classes.
Found 1317 files belonging to 9 classes.
Found 1328 files belonging to 9 classes.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
# Define the base_model

# Feature extraction with VGG16
import keras
import numpy as np

# 1. Prepare the base VGG16 pre-trained model
base_model: keras.applications.VGG16 = keras.applications.VGG16(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
import keras

# 1. Data augmentation constant (add the layer in runtime TPU usage)
data_augmentation = keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical"),
  keras.layers.RandomRotation(0.1),
  keras.layers.RandomZoom(0.1),
  keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
])

# 2. Freeze the base model to fine-tune only the top layers
base_model.trainable = False  # Freeze convolutional base

# 3. Create the model
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.02))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(9, activation='softmax')(x)
model_vgg16 = keras.Model(inputs, outputs)

model_vgg16.summary()
# 4. Compile the model
model_vgg16.compile(
  loss='sparse_categorical_crossentropy',
  optimizer=keras.optimizers.Adam(learning_rate=1e-4),
  metrics=['accuracy']
)

# 5. Create callback to save the best model
callbacks = [
  keras.callbacks.ModelCheckpoint(
    filepath=curr_dir / 'trash_classifier_preprocessed' / 'vgg16_finetuned.keras',
    save_best_only=True,
    monitor='val_loss',
  )
]

fine_tune_callbacks = [
  keras.callbacks.ModelCheckpoint(
    filepath=curr_dir / 'trash_classifier_preprocessed' / 'vgg16_finetuned_final.keras',
    save_best_only=True,
    monitor='val_loss',
  )
]

# 6. Train the model
history_vgg16 = model_vgg16.fit(
  train_dataset,
  epochs=40,
  validation_data=validation_dataset,
  callbacks=callbacks,
)

# 7. Evaluate the model
loss, accuracy = model_vgg16.evaluate(test_dataset)
print(f"Test accuracy: {accuracy:.2f}")


model_vgg16.load_weights(curr_dir / 'trash_classifier_preprocessed' / 'vgg16_finetuned.keras')

# After training initial layers (Fine Tuning)
base_model.trainable = True
# Optionally still freeze earlier layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

model_vgg16.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

# Fine-tune
history_finetune = model_vgg16.fit(
    train_dataset,
    epochs=40,
    validation_data=validation_dataset,
    callbacks=fine_tune_callbacks,
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ sequential[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ sequential[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 7, 7, 512) │ 14,714,688 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ global_average_p… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 512)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     65,664 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 128)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 9)         │      1,161 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,784,073 (56.40 MB)

 Trainable params: 14,782,793 (56.39 MB)

 Non-trainable params: 1,280 (5.00 KB)

Epoch 1/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 1548s 5s/step - accuracy: 0.1667 - loss: 6.2436 - val_accuracy: 0.1047 - val_loss: 9.4410
Epoch 2/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 181s 487ms/step - accuracy: 0.2635 - loss: 4.2580 - val_accuracy: 0.1107 - val_loss: 24.6932
Epoch 3/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 138s 490ms/step - accuracy: 0.3450 - loss: 3.0024 - val_accuracy: 0.2711 - val_loss: 4.7204
Epoch 4/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 144s 498ms/step - accuracy: 0.4336 - loss: 2.3470 - val_accuracy: 0.3366 - val_loss: 2.8701
Epoch 5/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 142s 497ms/step - accuracy: 0.4847 - loss: 1.9981 - val_accuracy: 0.4639 - val_loss: 1.9252
Epoch 6/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 136s 480ms/step - accuracy: 0.5280 - loss: 1.7719 - val_accuracy: 0.4202 - val_loss: 2.1956
Epoch 7/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 140s 474ms/step - accuracy: 0.5746 - loss: 1.5587 - val_accuracy: 0.3042 - val_loss: 4.0904
Epoch 8/40
282/282 ━━━━━━━━━━━━━━━━━━━━ 142s 475ms/step - accuracy: 0.5997 - 

In [ ]:
# Plot results
import matplotlib.pyplot as plt
acc = history_vgg16.history['accuracy']
val_acc = history_vgg16.history['val_accuracy']
loss = history_vgg16.history['loss']
val_loss = history_vgg16.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

print("Model train accuracy: ", acc)
print("Model val acc: ", val_acc)
print("Model loss: ", loss)
print("Model val loss: ", val_loss)

In [ ]:
import tensorflow as tf
import keras
from pathlib import Path

curr_dir: Path = Path('/content/drive/MyDrive/Colab Notebooks/Classifier/')

model = keras.models.load_model(curr_dir / 'trash_classifier_preprocessed' / 'vgg16_finetuned_final.keras')

results = model.evaluate(test_dataset)
print(results)


In [ ]:
print()